In [106]:
# Guide for Tuturial
#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/custom_batch_prediction_feature_filter.ipynb
#https://cloud.google.com/vertex-ai/docs/tutorials/train-tensorflow-bigquery
#https://cloud.google.com/vertex-ai/docs/tutorials/tabular-bq-prediction

In [107]:
import argparse
import os
from typing import Tuple, Optional

import pandas as pd
import numpy as np
import tensorflow as tf

from google.cloud import bigquery
from google.cloud import storage

from sklearn.model_selection import train_test_split

In [108]:
EPOCHS = 100
BATCH_SIZE = 32
TRAIN_STRATEGY = "single"


mean_and_std_json_file="incident_mean_and_std.json"

LABEL_COLUMN = "severity_name"
UNUSED_COLUMNS = ['severity_id','label_binary_severity','label_multi_severity']

model_dir='model'

In [109]:
# def load_data_bq(sql:str):
#  client_bq = bigquery.Client()
#  query_result=client_bq.query(sql)
#  df=query_result.to_dataframe()
#  return df

# def download_table(bq_table_uri: str):
#     # Remove bq:// prefix if present
#     bqclient= bigquery.Client()
#     prefix = "bq://"
#     if bq_table_uri.startswith(prefix):
#         bq_table_uri = bq_table_uri[len(prefix) :]

#     table = bigquery.TableReference.from_string(bq_table_uri)
#     rows = bqclient.list_rows(
#         table,
#     )
#     return rows.to_dataframe()

# df = download_table("pongthorn.SMartML.TrainEval_Incident_20230316")

def load_ml_data(data_path):
 df=pd.read_csv(data_path)
 df =df.drop(columns=UNUSED_COLUMNS)
 return df

root_path='../../data'    

dfAll=pd.read_csv(f"{root_path}/ML_Incident_20230316.csv",
                  usecols=['severity_name','sla','product_type','brand','service_type','incident_type'])

df_train = load_ml_data(f"{root_path}/train_incident.csv")
# val=train.copy()
df_validation=load_ml_data(f"{root_path}/validation_incident.csv")
# test =val.copy()
df_test =load_ml_data(f"{root_path}/test_incident.csv")

# sr_predict=df.iloc[-1,:]
# df=df.iloc[0:len(df)-1,:]


In [110]:
print(train.info())
train.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   severity_name              1714 non-null   object 
 1   sla                        1714 non-null   object 
 2   product_type               1714 non-null   object 
 3   brand                      1714 non-null   object 
 4   service_type               1714 non-null   object 
 5   incident_type              1714 non-null   object 
 6   open_to_close_hour         1714 non-null   float64
 7   response_to_resolved_hour  1714 non-null   float64
dtypes: float64(2), object(6)
memory usage: 107.2+ KB
None


,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour
1709,Major,8x5 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,47.966667,5.000000
1710,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,7.716667,4.683333
1711,Minor,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,18.000000,18.000000
1712,Major,24x7 4Hrs Resolution Time,Hardware,Oracle,Incident,System Board Failure,287.000000,285.733333
1713,Minor,24x7 4Hrs Resolution Time,Storage,NetApp,Incident,Hard Disk Drive Failure,8.983333,8.550000


In [111]:
print(dfAll.info())
dfAll.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   severity_name  2143 non-null   object
 1   sla            2143 non-null   object
 2   product_type   2143 non-null   object
 3   brand          2143 non-null   object
 4   service_type   2143 non-null   object
 5   incident_type  2143 non-null   object
dtypes: object(6)
memory usage: 100.6+ KB
None


,severity_name,sla,product_type,brand,service_type,incident_type
2138,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident
2139,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident
2140,Minor,24x7 4Hrs Response Time,Server,DELL,Incident,General Incident
2141,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure
2142,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,Maintenance System


In [112]:
list_label=dfAll[LABEL_COLUMN].unique()
print(list_label)

cate_sla=dfAll['sla'].unique()
print(cate_sla)

cate_productType=dfAll['product_type'].unique()
print(cate_productType)

cate_brand=dfAll['brand'].unique()
print(cate_brand)

cate_serviceType=dfAll['service_type'].unique()
print(cate_serviceType)

cate_incidentType=dfAll['incident_type'].unique()
print(cate_incidentType)


_CATEGORICAL_TYPES = {  
    LABEL_COLUMN:pd.api.types.CategoricalDtype(categories=list_label),
    "sla": pd.api.types.CategoricalDtype(categories=cate_sla),
    "product_type": pd.api.types.CategoricalDtype(categories=cate_productType),
    "brand": pd.api.types.CategoricalDtype(categories=cate_brand),
    "service_type": pd.api.types.CategoricalDtype(categories=cate_serviceType),
    "incident_type": pd.api.types.CategoricalDtype(categories=cate_incidentType),
}
#print(_CATEGORICAL_TYPES)

['Critical' 'Major' 'Minor' 'Cosmatic']
['24x7 4Hrs Response Time' '24x7 4Hrs Resolution Time'
 '24x7 6Hrs Resolution Time' '24x7 6Hrs Response Time'
 '8x5 4Hrs Response Time' '9x5xNBD 4Hrs Response Time']
['Server' 'Storage' 'Hardware' 'Software' 'Tape Library' 'Access Point'
 'Firewall' 'Other' 'Switch' 'Security']
['HPE' 'Oracle' 'VMWare' 'NetApp' 'YIP' 'DELL' 'Veeam' 'Cisco' 'Red Hat'
 'Palo Alto' 'Veritas' 'Trend Micro' 'F5' 'EMC' 'IBM' 'Nutanix'
 'Microsoft' 'Broadcom' 'CheckPoint' 'Fortinet' 'SAPB1' 'eUnite']
['Incident' 'Request']
['Network Card Failure' 'Memory Failure' 'Hard Disk Drive Failure'
 'General Incident' 'Software' 'Power Supply Failure'
 'Configuration Change' 'Upgrade Software' 'Network Adapter Failure'
 'Report' 'Battery Failure' 'System Board Failure' 'OS / Firmware'
 'Controller/Node Failure' 'Backup Failure' 'Maintenance System'
 'CPU Failure' 'Network Cable Failure' 'Other Failure'
 'Cache Battery Failure' 'Fan Failure']


In [113]:
def download_mean_and_std(mean_and_std_json_file):
    """Download mean and std for each column"""
    import json
    file_path=mean_and_std_json_file
    # bucket, file_path = extract_bucket_and_prefix_from_gcs_path(mean_and_std_json_file)
    # download_blob(bucket_name=bucket, source_blob_name=file_path, destination_file_name=file_path)
    
    with open(file_path, 'r') as file:
        return json.loads(file.read())

In [114]:
def preprocess(df):
    """Converts categorical features to numeric. Removes unused columns.

    Args:
      df: Pandas df with raw data

    Returns:
      df with preprocessed data
    """

    # Drop rows with NaN's
    df = df.dropna()

    # Convert integer valued (numeric) columns to floating point
    numeric_columns = df.select_dtypes(["int32", "float32", "float64"]).columns
    df[numeric_columns] = df[numeric_columns].astype("float32")

    # Convert categorical columns to numeric
    cat_columns = df.select_dtypes(["object"]).columns

    df[cat_columns] = df[cat_columns].apply(
        lambda x: x.astype(_CATEGORICAL_TYPES[x.name])
    )
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    return df

In [115]:
def standardize(df, mean_and_std):
    """Scales numerical columns using their means and standard deviation to get
    z-scores: the mean of each numerical column becomes 0, and the standard
    deviation becomes 1. This can help the model converge during training.

    Args:
      df: Pandas df

    Returns:
      Input df with the numerical columns scaled to z-scores
    """
    dtypes = list(zip(df.dtypes.index, map(str, df.dtypes)))
    # Normalize numeric columns.
    for column, dtype in dtypes:
        if dtype == "float32":
            df[column] -= mean_and_std[column]["mean"]
            df[column] /= mean_and_std[column]["std"]
    return df


In [116]:
def convert_dataframe_to_dataset(
    df_train,
    df_validation,
    mean_and_std
):

    df_train_x, df_train_y = df_train, df_train.pop(LABEL_COLUMN)
    df_validation_x, df_validation_y = df_validation, df_validation.pop(LABEL_COLUMN)

    # Join train_x and eval_x to normalize on overall means and standard
    # deviations. Then separate them again.
    all_x = pd.concat([df_train_x, df_validation_x], keys=["train", "eval"])
    all_x = standardize(all_x, mean_and_std)
    df_train_x, df_validation_x = all_x.xs("train"), all_x.xs("eval")

    y_train = np.asarray(df_train_y).astype("float32")
    y_validation = np.asarray(df_validation_y).astype("float32")

    # Convert to numpy representation
    x_train = np.asarray(df_train_x)
    x_test = np.asarray(df_validation_x)

    # Convert to one-hot representation
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(list_label))
    y_validation = tf.keras.utils.to_categorical(y_validation, num_classes=len(list_label))
    
    print(x_train.shape,y_train.shape, x_test.shape,y_validation.shape)
    
    # return   x_train,y_train, x_test,y_validation

    dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset_validation = tf.data.Dataset.from_tensor_slices((x_test, y_validation))
    
    return (dataset_train, dataset_validation)

In [117]:
def create_model(num_features,num_classes):
    # Create model
    Dense = tf.keras.layers.Dense
    model = tf.keras.Sequential(
        [
            Dense(
                32,
                activation=tf.nn.relu,
                input_dim=num_features,
            ),
            Dense(32, activation=tf.nn.relu),
            Dense(num_classes, activation=tf.nn.softmax),
        ]
    )
    
    # Compile Keras model
    # optimizer = tf.keras.optimizers.RMSprop(lr=0.001)
    model.compile(
        loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam'
    )
    
    return model

In [118]:
mean_and_std = download_mean_and_std(mean_and_std_json_file)
print(mean_and_std)

{'open_to_close_hour': {'mean': 95.19261960326722, 'std': 264.21816839738614}, 'response_to_resolved_hour': {'mean': 71.93835083625049, 'std': 224.20193515437524}}


In [119]:
df_train = preprocess(df_train)
df_validation = preprocess(df_validation)
#df_test = preprocess(df_test)
print(df_train.info())
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   severity_name              1714 non-null   int8   
 1   sla                        1714 non-null   int8   
 2   product_type               1714 non-null   int8   
 3   brand                      1714 non-null   int8   
 4   service_type               1714 non-null   int8   
 5   incident_type              1714 non-null   int8   
 6   open_to_close_hour         1714 non-null   float32
 7   response_to_resolved_hour  1714 non-null   float32
dtypes: float32(2), int8(6)
memory usage: 23.6 KB
None


,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour
0,1,1,3,8,0,3,1463.416626,1463.233276
1,1,2,1,3,0,2,5.916667,5.916667
2,1,0,3,2,0,6,0.883333,0.500000
3,1,1,1,0,0,2,113.849998,90.516670
4,2,0,3,2,0,3,17.116667,0.716667


In [120]:
dataset_train, dataset_validation = convert_dataframe_to_dataset(
  df_train, 
  df_validation, 
  mean_and_std
)
dataset_train = dataset_train.shuffle(len(df_train))


(1714, 7) (1714, 4) (214, 7) (214, 4)


In [121]:
model = create_model( num_features=dataset_train._flat_shapes[0].dims[0].value,num_classes=len(list_label))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                256       
                                                                 
 dense_7 (Dense)             (None, 32)                1056      
                                                                 
 dense_8 (Dense)             (None, 4)                 132       
                                                                 
Total params: 1,444
Trainable params: 1,444
Non-trainable params: 0
_________________________________________________________________


In [122]:
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_validation = dataset_validation.batch(BATCH_SIZE)

In [123]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)
history=model.fit(dataset_train, epochs=EPOCHS, validation_data=dataset_validation,batch_size=BATCH_SIZE,callbacks=[early_stopping])

Epoch 1/100
54/54 [==============================] - 1s 6ms/step - loss: 1.2965 - accuracy: 0.3553 - val_loss: 1.1655 - val_accuracy: 0.4860
Epoch 2/100
54/54 [==============================] - 0s 3ms/step - loss: 1.1436 - accuracy: 0.5292 - val_loss: 1.1122 - val_accuracy: 0.5374
Epoch 3/100
54/54 [==============================] - 0s 3ms/step - loss: 1.0869 - accuracy: 0.6033 - val_loss: 1.0632 - val_accuracy: 0.5748
Epoch 4/100
54/54 [==============================] - 0s 4ms/step - loss: 1.0302 - accuracy: 0.6079 - val_loss: 1.0008 - val_accuracy: 0.5467
Epoch 5/100
54/54 [==============================] - 0s 4ms/step - loss: 0.9697 - accuracy: 0.6097 - val_loss: 0.9367 - val_accuracy: 0.5514
Epoch 6/100
54/54 [==============================] - 0s 4ms/step - loss: 0.9142 - accuracy: 0.6120 - val_loss: 0.8875 - val_accuracy: 0.6215
Epoch 7/100
54/54 [==============================] - 0s 4ms/step - loss: 0.8725 - accuracy: 0.6167 - val_loss: 0.8638 - val_accuracy: 0.5888
Epoch 8/100
5

In [124]:
loss, accuracy = model.evaluate(dataset_validation)
print("Average Accuracy on Eveluation", accuracy)

7/7 [==============================] - 0s 3ms/step - loss: 0.7899 - accuracy: 0.6776
Average Accuracy on Eveluation 0.677570104598999


In [125]:
tf.saved_model.save(model, model_dir)

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets
